### 1. Create a Jupyter Notebook for Coursera Data Science Capstone Course Week 3 Peer Reviewed Assignment Clustering Neighborhoods in Toronto
   Mark Hanson

### 2. Scrape the Postal Code of Canada Wikipedia page

To obtain the data the Beautiful Soup module is used to scrape the webpage for postal codes and to transform the data into a pandas dataframe.

Start by importing the required modules.

In [55]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
import bs4
from bs4 import BeautifulSoup as bs

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Specify the Wikipedia URL, issue a request for the page, and store it as a BeautifulSoup object

In [2]:
# URL for Wikipedia page listing postal codes of Canada
url_wiki ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# request for Wikipedia page
resp = requests.get(url_wiki)

# store text part of response as beautifulsoup opbject
soup = bs(resp.text,'lxml')

### 3. Create and clean the dataframe

Select entire postal codes table and store it as a dataframe

In [3]:
# select the first table; Note - modifier :nth-of-type(1) limits the selection to just the first table on this page
table = soup.select('table:nth-of-type(1)')

# create a pandas dataframe by reading the table as html; Note - without [0] only postcode and borough show up
fas_df = pd.read_html(str(table))[0]

# declare first row as header; remove first row from dataframe; set header to be dataframe colums
header = fas_df.iloc[0]
fas_df = fas_df[1:]
fas_df.columns = header

In [4]:
#fas_df.head(20)

Clean the postal codes dataframe (remove Borough == 'Not assigned', Copy Borough to Neighbourhood if Neighbourhood == 'Not assigned', List all Neighbourhoods for given postal code on one line)

In [5]:
# print count by Borough and count of Borough Not assigned
#print(fas_df.groupby('Borough').count(), '\n')
#print(fas_df[fas_df.Borough != 'Not assigned'].count())

In [6]:
# update dataframe to keep rows were Borough is not equal to Not assigned i.e. remove Borough == 'Not assigned'
fas_df = fas_df[fas_df.Borough != 'Not assigned']

In [7]:
# reset index to assending numerical again after dropping some rows
fas_df = fas_df.reset_index(drop=True)

In [8]:
# print count by Neighbourhood
#print(fas_df[fas_df.Neighbourhood == 'Not assigned'].count(), '\n')

In [9]:
# Copy Borough to Neighbourhood if Neighbourhood equals 'Not assigned'
for index, row in fas_df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

In [10]:
# print count by Postcode
pcodes = fas_df.Postcode.unique()
print(len(pcodes), '\n')

#fas_df.groupby('Postcode').count()
#fas_df.groupby('Postcode').count()

103 



In [11]:
# List all Neighbourhoods for given postal code on one line
fas_df = fas_df.groupby(['Postcode']).agg({'Borough': 'first','Neighbourhood': ', '.join}).reset_index()
fas_df = fas_df[['Postcode','Borough','Neighbourhood']]

In [12]:
fas_df.shape

(103, 3)

In [13]:
fas_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### 4. Submit link to Notebook on my Github repo
Posted to Github and submitted link in Coursera. One notebook for all sections.

### 5. Get the latitude and longitude data for each neighborhood using Geocoder Python package in a while loop since the package is inconsistent or by using the csv file provided

Install and import geocoder package

In [14]:
# Install geocoder package with conda
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  16.46 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  31.63 MB/s


Initially had problems with install and import geocoder so used csv data provided instead - subsequently got geocoders installed

In [15]:
# declare URL for geographical coordinates of each Canadian postal code and read csv data into pandas dataframe
url_coord ='http://cocl.us/Geospatial_data'
coord_df = pd.read_csv(url_coord)

In [16]:
coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
# rename post column prior prior to merging with fas_df
coord_df.rename(index=str, columns={'Postal Code':'Postcode', 'Latitude':'Lat', 'Longitude':'Lng'}, inplace=True)

In [18]:
coord_df.shape

(103, 3)

In [19]:
fas_coord_df = pd.merge(fas_df, coord_df, how='left', on='Postcode', indicator=True)

### 6. Submit link to updated Notebook with latitude and longitude data on my Github repo
Dataframe with latitide and longitude data. Posted to Github and submitted link in Coursera. One notebook for all sections.

In [20]:
fas_coord_df.shape

(103, 6)

In [21]:
fas_coord_df

,Postcode,Borough,Neighbourhood,Lat,Lng,_merge
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,both
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,both
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,both
3,M1G,Scarborough,Woburn,43.770992,-79.216917,both
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,both
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,both
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,both
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,both
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,both
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,both


### 7. Explore and cluster the neighborhoods in Toronto.
You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 
Just make sure:
1.	to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
2.	to generate maps to visualize your neighborhoods and how they cluster together.


### Explanation of my analysis
For this portion of the assignment I used the Foursquare API to get the top venues in the postcodes with the Downtown string in the Borough name and compared the venues in each to see which postcodes are similar.

#### Install packages and set up mapping

In [22]:
# install folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  27.49 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  32.82 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  35.53 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  43.54 MB/s


In [23]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario, Canada are 43.653963, -79.387207.


In [24]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

In [25]:
#map_toronto

#### Use Foursquare to expore neighbourhoods

In [26]:
# The code was removed by Watson Studio for sharing.

Find neighbourhoods in Downtown Toronto

In [66]:
# explore around Union Station which is near where the Toronto Maple Leafs play so get lat & lng and index loc
downtown_area = fas_coord_df[fas_coord_df.Borough.str.contains('Downtown')]
downtown_area

,Postcode,Borough,Neighbourhood,Lat,Lng,_merge
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,both
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,both
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,both
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,both
54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,both
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,both
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,both
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,both
58,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,both
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,both


Select index 59 or postcode M5J and get its latitude and longitude information

In [28]:
# get lat long and names
lat = fas_coord_df.loc[59, 'Lat'] # neighborhood latitude value
lng = fas_coord_df.loc[59, 'Lng'] # neighborhood longitude value

nh_names = fas_coord_df.loc[59, 'Neighbourhood'] # neighborhood names

print('Latitude and longitude values of {} are {}, {}.'.format(nh_names, lat,lng))

Latitude and longitude values of Harbourfront East, Toronto Islands, Union Station are 43.6408157, -79.38175229999999.


Create the Foursquare API URL

In [30]:
# set radius to 500 meteres and limit search to 100 venues
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limmit={}'.format(
    CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=USIQRFGYSZF4WCVF3ES5N2LB2GV01TUKYRRAVVXVBXI4OHBH&client_secret=EK4HAZFNQQDVCOJYSCF2XCYLGWVNVIENU2ITX30KKKBIWAR4&ll=43.6408157,-79.38175229999999&v=20180605&radius=500&limmit=100'

In [36]:
# store results from foursquare api
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cad3e6d9fb6b712bfd2824c'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bfaa3494a67c928d08528cf-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
          'suffix': '.png'},
         'id': '4f2a25ac4b909258e854f55f',
         'name': 'Neighborhood',
         'pluralName': 'Neighborhoods',
         'primary': True,
         'shortName': 'Neighborhood'}],
       'id': '4bfaa3494a67c928d08528cf',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 167,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.639525632239106,
          'lng': -79.38068838052389}],
        'lat': 43.639

Copy funtions from New York lab to get category type and create nearby venues dataframe

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
# get venues from response
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Harbourfront,Neighborhood,43.639526,-79.380688
1,Roundhouse Park,Park,43.641745,-79.384279
2,iQ Food Co,Salad Place,43.642851,-79.382081
3,BeaverTails,Bakery,43.639899,-79.380197
4,Longo's Maple Leaf Square,Supermarket,43.642517,-79.381393


Explore Downtown Toronto area

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Lat', 
                  'Postcode Lng', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Write code to run the above function on each postcode and create a new dataframe called *downtown_venues*

In [67]:
downtown_venues = getNearbyVenues(names=downtown_area['Postcode'], latitudes=downtown_area['Lat'], longitudes=downtown_area['Lng'])

M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5S
M5T
M5V
M5W
M5X
M6G


In [68]:
print(downtown_venues.shape)
downtown_venues.head()

(1285, 7)


,Postcode,Postcode Lat,Postcode Lng,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4W,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,M4W,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,M4W,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,M4W,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,M4X,43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [69]:
downtown_venues.groupby('Postcode').count()

,Postcode Lat,Postcode Lng,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M4W,4,4,4,4,4,4
M4X,43,43,43,43,43,43
M4Y,89,89,89,89,89,89
M5A,48,48,48,48,48,48
M5B,100,100,100,100,100,100
M5C,100,100,100,100,100,100
M5E,55,55,55,55,55,55
M5G,84,84,84,84,84,84
M5H,100,100,100,100,100,100


In [70]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 204 uniques categories.


Analyze each postcode

In [78]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
downtown_onehot['Postcode'] = downtown_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
downtown_onehot.head()

,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4W,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,M4X,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
downtown_onehot.shape

(1285, 205)

In [73]:
downtown_grouped = downtown_onehot.groupby('Postcode').mean().reset_index()

In [74]:
downtown_grouped.shape

(18, 205)

In [75]:
num_top_venues = 5

for postcode in downtown_grouped['Postcode']:
    print("----"+postcode+"----")
    temp = downtown_grouped[downtown_grouped['Postcode'] == postcode].T.reset_index()   # .T transposes dataframe
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4W----
               venue  freq
0               Park  0.50
1         Playground  0.25
2              Trail  0.25
3  Accessories Store  0.00
4        Music Venue  0.00


----M4X----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.07
2         Pizza Place  0.05
3              Bakery  0.05
4  Italian Restaurant  0.05


----M4Y----
                 venue  freq
0          Coffee Shop  0.07
1  Japanese Restaurant  0.07
2              Gay Bar  0.04
3     Sushi Restaurant  0.04
4           Restaurant  0.03


----M5A----
         venue  freq
0  Coffee Shop  0.17
1         Café  0.06
2       Bakery  0.06
3         Park  0.06
4          Pub  0.06


----M5B----
                       venue  freq
0                Coffee Shop  0.08
1             Clothing Store  0.07
2             Cosmetics Shop  0.04
3                       Café  0.04
4  Middle Eastern Restaurant  0.03


----M5C----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06
2     

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each postcode.

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['Postcode'] = downtown_grouped['Postcode']

for ind in np.arange(downtown_grouped.shape[0]):
    postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

postcode_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
1,M4X,Coffee Shop,Restaurant,Pub,Italian Restaurant,Café,Bakery,Pizza Place,Bank,Gastropub,Liquor Store
2,M4Y,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Yoga Studio,Gastropub,Fast Food Restaurant,Gym
3,M5A,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Brewery,Restaurant
4,M5B,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Japanese Restaurant,Diner,Pizza Place


### Cluster Postcodes

In [80]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [81]:
# add clustering labels
postcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_area

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(postcode_venues_sorted.set_index('Postcode'), on='Postcode')   #this should be on Postcode

downtown_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Lat,Lng,_merge,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,both,2,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,both,1,Coffee Shop,Restaurant,Pub,Italian Restaurant,Café,Bakery,Pizza Place,Bank,Gastropub,Liquor Store
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,both,1,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Yoga Studio,Gastropub,Fast Food Restaurant,Gym
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,both,1,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Brewery,Restaurant
54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,both,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Japanese Restaurant,Diner,Pizza Place


In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Lat'], downtown_merged['Lng'], downtown_merged['Postcode'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

### Cluster 0 Urban Living

In [85]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,_merge,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Downtown Toronto,both,0,Café,Grocery Store,Park,Italian Restaurant,Restaurant,Coffee Shop,Convenience Store,Nightclub,Diner,Baby Store


### Cluster 1 Coffee / Bar / Restaurant

In [86]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,_merge,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Downtown Toronto,both,1,Coffee Shop,Restaurant,Pub,Italian Restaurant,Café,Bakery,Pizza Place,Bank,Gastropub,Liquor Store
52,Downtown Toronto,both,1,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Yoga Studio,Gastropub,Fast Food Restaurant,Gym
53,Downtown Toronto,both,1,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Brewery,Restaurant
54,Downtown Toronto,both,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Japanese Restaurant,Diner,Pizza Place
55,Downtown Toronto,both,1,Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Bakery,Gastropub,Cosmetics Shop,Italian Restaurant,Clothing Store
56,Downtown Toronto,both,1,Coffee Shop,Cocktail Bar,Pub,Steakhouse,Farmers Market,Café,Cheese Shop,Restaurant,Seafood Restaurant,Bakery
57,Downtown Toronto,both,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Bar,Burger Joint,Spa,Chinese Restaurant,Salad Place
58,Downtown Toronto,both,1,Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,American Restaurant,Restaurant,Sushi Restaurant,Hotel,Bakery
59,Downtown Toronto,both,1,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Fried Chicken Joint,Scenic Lookout,Bakery,Pizza Place,Brewery
60,Downtown Toronto,both,1,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Italian Restaurant,Deli / Bodega,Gastropub,Burger Joint,Steakhouse


### Cluster 2 Park

In [87]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,_merge,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,both,2,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


### Cluster 3  Airport / Harbor

In [88]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,_merge,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,both,3,Airport Terminal,Airport Service,Airport Lounge,Airport Gate,Boat or Ferry,Plane,Harbor / Marina,Airport Food Court,Airport,Sculpture Garden


### Cluster 4 Campus

In [89]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,_merge,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Downtown Toronto,both,4,Café,Bakery,Theater,Japanese Restaurant,Bar,Bookstore,Restaurant,Beer Bar,Comfort Food Restaurant,Sandwich Place
